# Stanford CME 241 (Winter 2024) - Assignment 3

**Due: Jan 29 @ 11:59pm Pacific Time on Gradescope.**

Assignment instructions:
- **Please solve questions 1 and 2, and choose one of questions 3 or 4.**
- Empty code blocks are for your use. Feel free to create more under each section as needed.

Submission instructions:
- When complete, fill out your publicly available GitHub repo file URL and group members below, then export or print this .ipynb file to PDF and upload the PDF to Gradescope.

*Link to this ipynb file in your public GitHub repo (replace below URL with yours):* 

https://github.com/HenriqueMonteiro35/RL-book/blob/master/assignment3.ipynb

*Group members (replace below names with people in your group):* 
- Arantxa Ramos del Valle - aramosv
- Henrique Bittencourt Netto Monteiro - hbnm

## Imports

In [1]:
from typing import Optional
from dataclasses import dataclass
import numpy as np

## Question 1
**Analytic Optimal Actions and Cost.** 
Consider a continuous-states, continuous-actions, discrete-time, non-terminating MDP with state space as $\mathbb{R}$ and action space as $\mathbb{R}$. When in state $s\in \mathbb{R}$, upon taking action $a\in \mathbb{R}$, one transitions to next state $s' \in \mathbb{R}$ according to a normal distribution $s' \sim \mathcal{N}(s, \sigma^2)$ for a fixed variance $\sigma^2 \in \mathbb{R}^+$. The corresponding cost associated with this transition is $e^{as'}$, i.e., the cost depends on the action $a$ and the state $s'$ one transitions to. The problem is to minimize the infinite-horizon **Expected Discounted-Sum of Costs** (with discount factor $\gamma < 1$). For this assignment, solve this problem just for the special case of $\gamma = 0$ (i.e., the myopic case) using elementary calculus. Derive an analytic expression for the optimal action in any state and the corresponding optimal cost.


**ANSWER:** since we are in the myopic case ($\gamma = 0$), we just need to find a policy to minimize the expected cost of a single transaction. Since the cost is $e^{as'}$ and $s'$ is normally distributed, we can compute the expected cost $E[C]$ analytically,

\begin{align}
E[C] &= E[e^{as'}] = \int_{\mathbb{R}} \exp \left( a s' \right) \frac{1}{\sqrt{2 \sigma^2 \pi}} \exp \left( \frac{-(s' - s)^2}{2 \sigma^2} \right) ds' \nonumber \\
&= \frac{1}{\sqrt{2 \sigma^2 \pi}} \int_{\mathbb{R}} \exp \left( \frac{-s'^2 + 2 s s' - s^2 + 2 a \sigma^2 s'}{2 \sigma^2} \right) ds' \nonumber \\
&= \frac{1}{\sqrt{2 \sigma^2 \pi}} \exp \left( \frac{- s^2}{2 \sigma^2} \right)
\int_{\mathbb{R}} \exp \left( \frac{-s'^2 + 2 s' (s + a \sigma^2)}{2 \sigma^2} \right) ds' \nonumber \\
&= \frac{1}{\sqrt{2 \sigma^2 \pi}} \exp \left( \frac{- s^2}{2 \sigma^2} + \frac{(s + a \sigma^2)^2}{2 \sigma^2} \right)
\int_{\mathbb{R}} \exp \left( \frac{-s'^2 + 2 s' (s + a \sigma^2) - (s + a \sigma^2)^2}{2 \sigma^2} \right) ds' \nonumber \\
&= \exp \left( \frac{2 a s \sigma^2 + a^2 \sigma^4}{2 \sigma^2} \right) \left( \frac{1}{\sqrt{2 \sigma^2 \pi}}
\int_{\mathbb{R}} \exp \left( \frac{-(s' - (s + a \sigma^2))^2}{2 \sigma^2} \right) ds' \right) \nonumber \\
&= \exp \left( \frac{2 a s + a^2 \sigma^2}{2} \right)
\end{align}

in which in the fourth line we completed squares and in the fifth we used the fact that the expression in parenthesis was the integral of a Gaussian PDF.

We now note that the exponential is a strictly monotically increasing function, and the term $\frac{2 a s + a^2 \sigma^2}{2}$ is quadratic in $a$ with positive second derivative. Therefore, for any given $s$, minimizing the expected cost $E[C]$ with respect to the action $a$ is a convex problem with unique minimizer. We thus find this minimizer by differentiating with respect to $a$ and equating to zero,

$$
\left( s + a \sigma^2 \right) \exp \left( \frac{2 a s + a^2 \sigma^2}{2} \right) = 0 \implies \left( s + a \sigma^2 \right) = 0
$$
$$
a = -\frac{s}{\sigma^2}
$$

in which we used the fact that the exponential is never zero for any real number. We now compute the optimal cost $C^*$,

$$
C^* = \exp \left( -\frac{s}{\sigma^2} s + \frac{s^2}{\sigma^4} \frac{\sigma^2}{2} \right)
$$

$$
C^* = \exp \left( - \frac{s^2}{2 \sigma^2} \right)
$$

Therefore, in the myopic case, given any state $s$, the optimal action is $a^* = -\frac{s}{\sigma^2}$ and the optimal cost is $C^* = \exp \left( - \frac{s^2}{2 \sigma^2} \right)$.

## Question 2
**Manual Value Iteration.** 
Consider a simple MDP with $\mathcal{S} = \{s_1, s_2, s_3\}, \mathcal{T} =\{s_3\}, \mathcal{A} = \{a_1, a_2\}$. The State Transition Probability function
$$\mathcal{P}: \mathcal{N} \times \mathcal{A} \times \mathcal{S} \rightarrow [0, 1]$$
is defined as:
$$\mathcal{P}(s_1, a_1, s_1) = 0.2, \mathcal{P}(s_1, a_1, s_2) = 0.6, \mathcal{P}(s_1, a_1, s_3) = 0.2$$
$$\mathcal{P}(s_1, a_2, s_1) = 0.1, \mathcal{P}(s_1, a_2, s_2) = 0.2, \mathcal{P}(s_1, a_2, s_3) = 0.7$$
$$\mathcal{P}(s_2, a_1, s_1) = 0.3, \mathcal{P}(s_2, a_1, s_2) = 0.3, \mathcal{P}(s_2, a_1, s_3) = 0.4$$
$$\mathcal{P}(s_2, a_2, s_1) = 0.5, \mathcal{P}(s_2, a_2, s_2) = 0.3, \mathcal{P}(s_2, a_2, s_3) = 0.2$$
The Reward Function 
$$\mathcal{R}: \mathcal{N} \times \mathcal{A} \rightarrow \mathbb{R}$$
is defined as:
$$\mathcal{R}(s_1, a_1) = 8.0, \mathcal{R}(s_1, a_2) = 10.0$$
$$\mathcal{R}(s_2, a_1) = 1.0, \mathcal{R}(s_2, a_2) = -1.0$$
Assume discount factor $\gamma = 1$.

Your task is to determine an Optimal Deterministic Policy **by manually working out** (not with code) simply the first two iterations of Value Iteration algorithm. 

- Initialize the Value Function for each state to be it's $\max$ (over actions) reward, i.e., we initialize the Value Function to be $v_0(s_1) = 10.0, v_0(s_2) = 1.0, v_0(s_3) = 0.0$. Then manually calculate $q_k(\cdot, \cdot)$ and $v_k(\cdot)$ from $v_{k - 1}( \cdot)$ using the Value Iteration update, and then calculate the greedy policy $\pi_k(\cdot)$ from $q_k(\cdot, \cdot)$ for $k = 1$ and $k = 2$ (hence, 2 iterations).
- Now argue that $\pi_k(\cdot)$ for $k > 2$ will be the same as $\pi_2(\cdot)$. Hint: You can make the argument by examining the structure of how you get $q_k(\cdot, \cdot)$ from $v_{k-1}(\cdot)$. With this argument, there is no need to go beyond the two iterations you performed above, and so you can establish $\pi_2(\cdot)$ as an Optimal Deterministic Policy for this MDP.

**Answer:**

In every iteration, the value function state-action is calculated as follows:

$${Q_{i+1}}(s,a)= \mathcal{R}(s,a)+\gamma\sum_{s'\in\mathcal{N}}\mathcal{P}(s,a,s')\cdot V_i(s')\ for\ all\ s\in\mathcal{N}\ for\ all\ a\in\mathcal{A}$$


The value function is calculated:

$${V_{i+1}}= B^*({V_{i}})(s)\ for\ all\ s\in\mathcal{N}$$
where,
$$B^*({V_{i}})(s)=\max_{a\in\mathcal{A}}\{\mathcal{R}(s,a)+\gamma\sum_{s'\in\mathcal{N}}\mathcal{P}(s,a,s')\cdot V(s')\}\ for\ all\ s\in\mathcal{N}$$

Or what is the same:

$${V_{i+1}}= \max_{a\in\mathcal{A}}{Q_{i+1}}(s,a)$$

Then the greedy policy is calculated, as follows:

$${\pi_{i+1}}= \arg\max_{a\in\mathcal{A}}{Q_{i+1}}(s,a)$$





We are initializing the value functions as: $v_0(s_1) = 10.0, v_0(s_2) = 1.0, v_0(s_3) = 0.0$

In the first iteration ($k=1$):
For the state $s_1$:

- $q_1(s_1,a_1)=8.0+1\cdot(0.2\cdot10.0+0.6\cdot1.0+0.2\cdot0.0)=10.6$
- $q_1(s_1,a_2)=10.0+1\cdot(0.1\cdot10.0+0.2\cdot1.0+0.7\cdot0.0)=11.2$


$$v_1(s_1)=\max\{8.0+1\cdot(0.2\cdot10.0+0.6\cdot1.0+0.2\cdot0.0),10.0+1\cdot(0.1\cdot10.0+0.2\cdot1.0+0.7\cdot0.0)\}=\max\{10.6,11.2\}=11.2$$

From this, the greedy policy is $\pi_1(s_1)=a_2$


For the state $s_2$:

- $q_1(s_2,a_1)=1.0+1\cdot(0.3\cdot10.0+0.3\cdot1.0+0.4\cdot0.0)=4.3$
- $q_1(s_2,a_2)=-1.0+1\cdot(0.5\cdot10.0+0.3\cdot1.0+0.2\cdot0.0)=4.3$


$$v_1(s_2)=\max\{1.0+1\cdot(0.3\cdot10.0+0.3\cdot1.0+0.4\cdot0.0),-1.0+1\cdot(0.5\cdot10.0+0.3\cdot1.0+0.2\cdot0.0)\}=\max\{4.3,4.3\}=4.3$$

From this, the greedy policy is $\pi_1(s_2)=a_2$


For the state $s_3$, $v_1(s_3)=0.0$ since it is a terminal state.


For $k=2$:
For the state $s_1$:

- $q_2(s_1,a_1)=8.0+1\cdot(0.2\cdot11.2+0.6\cdot4.3+0.2\cdot0.0)=12.82$
- $q_2(s_1,a_2)=10.0+1\cdot(0.1\cdot11.2+0.2\cdot4.3+0.7\cdot0.0)=11.98$


$$v_2(s_1)=\max\{8.0+1\cdot(0.2\cdot11.2+0.6\cdot4.3+0.2\cdot0.0),10.0+1\cdot(0.1\cdot11.2+0.2\cdot4.3+0.7\cdot0.0)\}=\max\{12.82,11.98\}=12.82$$

From this, the greedy policy is $\pi_2(s_1)=a_1$


For the state $s_2$:

- $q_2(s_2,a_1)=1.0+1\cdot(0.3\cdot11.2+0.3\cdot4.3+0.4\cdot0.0)=5.65$
- $q_2(s_2,a_2)=-1.0+1\cdot(0.5\cdot11.2+0.3\cdot4.3+0.2\cdot0.0)=5.89$


$$v_2(s_2)=\max\{1.0+1\cdot(0.3\cdot11.2+0.3\cdot4.3+0.4\cdot0.0),-1.0+1\cdot(0.5\cdot11.2+0.3\cdot4.3+0.2\cdot0.0)\}=\max\{5.65,5.89\}=5.89$$

From this, the greedy policy is $\pi_2(s_2)=a_2$


For the state $s_3$, $v_2(s_3)=0.0$ since it is a terminal state.

The greedy policy $\pi_k(s)$ for $k>2$ is the same policy for every state $s$ as the greedy policy $\pi_2(s)$. Note that action $a_1$ at state $s_1$ has equal or higher probability to transition to each of $s_1$ and $s_2$ than $a_2$, and for state $s_2$, action $a_2$ has the same property compared to $a_1$. Given that and the fact that the value function is monotonically increasing at each iteration, these actions must be optimal as the gap in value between choosing them versus the alternative can only remain constant or widen at future iterations.

Formally, for any state $k$, we have

- $q_k(s_1, a_1) = \mathcal{R}(s_1, a_1) + 0.2 v_{k-1}(s_1) + 0.6 v_{k-1}(s_1)$
- $q_k(s_1, a_2) = \mathcal{R}(s_1, a_2) + 0.1 v_{k-1}(s_1) + 0.2 v_{k-1}(s_1)$

- $q_k(s_2, a_1) = \mathcal{R}(s_2, a_1) + 0.3 v_{k-1}(s_1) + 0.3 v_{k-1}(s_1)$
- $q_k(s_2, a_2) = \mathcal{R}(s_2, a_2) + 0.5 v_{k-1}(s_1) + 0.3 v_{k-1}(s_1)$

Then, the differences in value between taking each action is

- $q_k(s_1, a_1) - q_k(s_1, a_2) = (\mathcal{R}(s_1, a_1) - \mathcal{R}(s_1, a_2)) + 0.1 v_{k-1}(s_1) + 0.4 v_{k-1}(s_2)$
- $q_k(s_2, a_2) - q_k(s_2, a_1) = (\mathcal{R}(s_2, a_2) - \mathcal{R}(s_2, a_1)) + 0.2 v_{k-1}(s_1)$

Finally,

- $(q_k(s_1, a_1) - q_k(s_1, a_2)) - (q_2(s_1, a_1) - q_2(s_1, a_2)) = 0.1 (v_{k-1}(s_1) - v_{1}(s_1)) + 0.4 (v_{k-1}(s_2) - v_{1}(s_2))$
- $(q_k(s_2, a_2) - q_k(s_2, a_1)) - (q_2(s_2, a_2) - q_2(s_2, a_1)) = 0.2 (v_{k-1}(s_1) - v_{1}(s_1))$

Since the value function is monotonically increasing, the two above expressions are nonnegative for any $k > 2$, which means that for every possible iteration beyond $k = 2$, the values associated with $a_1$ at state $s_1$ and $a_2$ at state $s_2$ are higher than taking the alternative action,

- $(q_k(s_1, a_1) - q_k(s_1, a_2)) \geq (q_2(s_1, a_1) - q_2(s_1, a_2)) > 0 \text{ } \forall k > 2$
- $(q_k(s_2, a_2) - q_k(s_2, a_1)) \geq (q_2(s_2, a_2) - q_2(s_2, a_1)) > 0 \text{ } \forall k > 2$

 so the greedy policy will never change from $\pi_2(\cdot)$, and one does not need to go beyond iteration $k = 2$. Policy $\pi_2(\cdot)$ must be optimal for this MDP.



<!-- Having a discount factor of 1 makes the future rewards as valuable as present rewards. The value functions are positive. Therefore, the action that makes less likely to reach the terminal state ($s_3$) is the greedy. -->

## Question 3

**Job-Hopping and Wages-Utility-Maximization.** 
You are a worker who starts every day either employed or unemployed. If you start your day employed, you work on your job for the day (one of $n$ jobs, as elaborated later) and you get to earn the wage of the job for the day. However, at the end of the day, you could lose your job with probability $\alpha \in [0,1]$, in which case you start the next day unemployed. If at the end of the day, you do not lose your job (with probability $1-\alpha$), then you will start the next day with the same job (and hence, the same daily wage). On the other hand, if you start your day unemployed, then you will be randomly offered one of $n$ jobs with daily wages $w_1, w_2, \ldots w_n \in \mathbb{R}^+$ with respective job-offer probabilities $p_1, p_2, \ldots p_n \in [0,1]$ (with $\sum_{i=1}^n p_i = 1$). You can choose to either accept or decline the offered job. If you accept the job-offer, your day progresses exactly like the **employed-day** described above (earning the day's job wage and possibly (with probability $\alpha$) losing the job at the end of the day). However, if you decline the job-offer, you spend the day unemployed, receive the unemployment wage $w_0 \in \mathbb{R}^+$ for the day, and start the next day unemployed. The problem is to identify the optimal choice of accepting or rejecting any of the job-offers the worker receives, in a manner that maximizes the infinite-horizon **Expected Discounted-Sum of Wages Utility**. Assume the daily discount factor for wages (employed or unemployed) is $\gamma \in [0,1)$. Assume Wages Utility function to be $U(w) = \log(w)$ for any wage amount $w \in \mathbb{R}^+$. So you are looking to maximize
$$\mathbb{E}[\sum_{u=t}^\infty \gamma^{u-t} \cdot \log(w_{i_u})]$$
at the start of a given day $t$ ($w_{i_u}$ is the wage earned on day $u$, $0\leq i_u \leq n$ for all $u\geq t$).

- Express with clear mathematical notation the state space, action space, transition function, reward function, and write the Bellman Optimality Equation customized for this MDP.
- You can solve this Bellman Optimality Equation (hence, solve for the Optimal Value Function and the Optimal Policy) with a numerical iterative algorithm (essentially a Dynamic Programming algorithm customized to this problem). Write Python code for this numerical algorithm. Clearly define the inputs and outputs of your algorithm with their types (int, float, List, Mapping etc.). For this problem, don't use any of the MDP/DP code from the git repo, write this customized algorithm from scratch.

**ANSWER:** we can define the state space as

$$
\mathcal{S} = \{s_1, s_2, \dots, s_n, \varsigma_1, \varsigma_2, \dots, \varsigma_n \}
$$

in which the states $s_i$ refer to starting the day employed at job $i$ while $\varsigma_i$ refers to starting the day unemployed with an offer from job $i$.

We can define the action space as

$$
\mathcal{A} = \{a_{w}, a_{r}\}
$$

in which $a_w$ is to work for the given job (accepting the offer if unemployed) and $a_r$ is to reject the offer if unemployed. Clearly,

$$
\mathcal{A} | s_i = \{a_{w}\} \text{ and } \mathcal{A} | \varsigma_i = \{a_{w}, a_r\}
$$

since the option to reject and not work is only available if you start the day unemployed.

The transition function can be defined as

$$
\mathcal{P}(s_i | s_i, a_w) = 1 - \alpha
$$
$$
\mathcal{P}(\varsigma_j | s_i, a_w) = \alpha p_j
$$

$$
\mathcal{P}(s_i | \varsigma_i, a_w) = 1 - \alpha
$$
$$
\mathcal{P}(\varsigma_j | \varsigma_i, a_w) = \alpha p_j
$$

$$
\mathcal{P}(\varsigma_j | \varsigma_i, a_r) = p_j
$$

in which all unspecified probabilities are zero.

The reward function can be defined as

$$
\mathcal{R}(s_i, a_w) = w_i
$$

$$
\mathcal{R}(\varsigma_i, a_w) = w_i
$$

$$
\mathcal{R}(\varsigma_i, a_r) = w_0
$$

in which all unspecified rewards are zero.

To write Bellman's Optimality Equation for this MDP, we start with the definition for some state $s \in \mathcal{S}$,

$$
V^*(s) = \max_{a \in \mathcal{A}} \{  \mathcal{R}(s, a) + \gamma \sum_{s' \in \mathcal{N}} \mathcal{P}(s, a, s') V^*(s') \}
$$

For $s \in \{s_1, s_2, \dots, s_n\}$, the only action is $a_w$, so

$$
V^*(s_i) = \max_{a \in \mathcal{A}} \{  \log(w_i) + \gamma (1 - \alpha) V^*(s_i) + \gamma \alpha \sum_{j = 1}^{n} p_j V^*(\varsigma_j) \}
$$

$$
V^*(s_i) =  \log(w_i) + \gamma (1 - \alpha) V^*(s_i) + \gamma \alpha \sum_{j = 1}^{n} p_j V^*(\varsigma_j)
$$

$$
V^*(s_i) =  \frac{\log(w_i) + \gamma \alpha \sum_{j = 1}^{n} p_j V^*(\varsigma_j)}{1 - \gamma (1 - \alpha)}
$$

Now for $s \in \{\varsigma_1, \varsigma_2, \dots, \varsigma_n\}$, the two actions are available, but note that $a_w$ reduces problem to the same as above,

$$
V^*(\varsigma_i) = \max_{a \in \mathcal{A}} \{  V^*(s_i), \log(w_0) + \gamma \sum_{j = 1}^{n} p_j V^*(\varsigma_j) \}
$$

$$
V^*(\varsigma_i) = \max \left \{  \frac{\log(w_i) + \gamma \alpha \sum_{j = 1}^{n} p_j V^*(\varsigma_j)}{1 - \gamma (1 - \alpha)}, \log(w_0) + \gamma \sum_{j = 1}^{n} p_j V^*(\varsigma_j) \right \}
$$

We provided above a couple different formulations for Bellman's Optimality Equation for both states in which you start employed and unemployed.

Interestingly, the optimal values (and optimal actions) for the unemployed states $\{\varsigma_i\}_{i=1}^n$ do not depend on those of the employed states $\{s_i\}_{i=1}^n$. This finding makes sense since there is a unique action available on the employed states, so there is no policy to be optimized for those states.
<!-- Therefore, in our implementation of a DP numerical algorithm, we will omit the employed states and use the Bellman's Optimality Equation as defined in the last equation above. -->

In [2]:
def format_policy(pi):
    assert(len(pi) % 2 == 0)
    n = len(pi) // 2
    assert(pi[:n].all())
    return (n*["Work"]) + ["Accept" if accept else "Reject" for accept in pi[n:]]

In [3]:
@dataclass
class Q3():
    alpha: float
    gamma: float
    unemp_wage: float
    wages: np.array
    probabilities: np.array

    pi: Optional[np.array] = None
    V: Optional[np.array] = None
    eps: Optional[float] = 1e-10

    n: int = None

    def __post_init__(self):
        assert(len(self.wages.shape) == 1 and len(self.probabilities.shape) == 1)
        assert(self.wages.shape == self.probabilities.shape)
        assert(np.abs(1 - self.probabilities.sum()) < self.eps)

        self.n = len(self.wages)
        if self.pi is None:
            self.pi = np.ones(2*self.n).astype(bool)
        assert(self.pi[:self.n].all())

        if self.V is None:
            self.V = np.zeros(2*self.n)
        self.V = self.value_iteration()

    # Reward matrix R
    def R(self, pi=None) -> np.array:
        if pi is None:
            pi = self.pi
        return np.array([self.get_reward(i, action) for (i, action) in enumerate(pi)])

    # Transition probability matrix P
    def P(self, pi=None) -> np.array:
        if pi is None:
            pi = self.pi
        # employed -> employed
        top_left = (1 - self.alpha) * np.eye(self.n)
        # employed -> unemployed
        top_right = np.repeat([self.alpha * self.probabilities], self.n, axis=0)
        # unemployed -> employed
        bottom_left = np.diag([(1 - self.alpha) * int(accept) for accept in pi[self.n:]])
        # unmployed -> unemployed
        bottom_right = [row if accept else row/self.alpha for (row, accept) in zip(top_right, pi[self.n:])]

        top = np.concatenate([top_left, top_right], axis=1)
        bottom = np.concatenate([bottom_left, bottom_right], axis=1)
        P = np.concatenate([top, bottom], axis=0)

        assert(P.shape == (2*self.n, 2*self.n))
        for row in P:
            assert(np.abs(1 - np.sum(row)) < self.eps)
        return P

    def get_reward(self, state: int, accept: bool) -> float:
        # UNEMPLOYED AND REJECTED
        if state >= self.n and not accept:
            return np.log(self.unemp_wage)
        else:
            return np.log(self.wages[state % self.n])

    def get_updated_V(self, V=None, pi=None) -> np.array:
        if V is None:
            V = self.V
        return self.R(pi=pi) + self.gamma * np.matmul(self.P(pi=pi), V)

    # Iterative algo to compute value vector from Bellman's operator
    def value_iteration(self, pi=None) -> np.array:
        V = self.V.copy()
        while np.linalg.norm(self.get_updated_V(V, pi=pi) - V, ord=np.inf) > self.eps:
            V = self.get_updated_V(V, pi=pi)
        return V

    # Greedy algo to get better policy
    def greedy_iteration(self) -> None:
        new_pi = self.pi.copy()
        for i in range(self.n, 2*self.n):
            test_pi = self.pi.copy()
            test_pi[i] = not self.pi[i]
            if self.value_iteration(pi=test_pi)[i] > self.V[i]:
                new_pi[i] = test_pi[i]
        self.pi = new_pi

    # Run value iteration and greedy iteration in a loop until convergence
    def policy_iteration(self) -> None:
        old_pi = None
        while not np.array_equal(old_pi, self.pi):
            old_pi = self.pi.copy()
            self.V = self.value_iteration()
            self.greedy_iteration()
        print(f"Optimal value:\n{self.V}")
        print(f"\nOptimal policy:\n{format_policy(self.pi)}")
        print(f"\nFinal R vector:\n{self.R()}")
        print(f"\nFinal P matrix:\n{self.P()}")

In [4]:
n = 5
alpha = 0.1
gamma = 0.9
unemp_wage = 1.0
wages = 2*np.random.random(n)
probabilities = np.ones(n)/n

start_pi = np.append(np.ones(n), np.zeros(n)).astype(bool)
start_V = np.zeros(2*n)

q3 = Q3(alpha, gamma, unemp_wage, wages, probabilities, start_pi, start_V)

print(f"Unemployment wage: {q3.unemp_wage}")
print(f"\nWages:\n{q3.wages}")
print(f"\nInitial value:\n{q3.V}")
print(f"\nInitial policy:\n{format_policy(q3.pi)}")
print(f"\nInitial R vector:\n{q3.R()}")
print(f"\nInitial P matrix:\n{q3.P()}")

Unemployment wage: 1.0

Wages:
[1.21615405 0.65993361 1.6378637  0.77585731 1.86649041]

Initial value:
[ 1.02996559 -2.18745283  2.59680405 -1.33571927  3.2845257   0.
  0.          0.          0.          0.        ]

Initial policy:
['Work', 'Work', 'Work', 'Work', 'Work', 'Reject', 'Reject', 'Reject', 'Reject', 'Reject']

Initial R vector:
[ 0.19569346 -0.41561604  0.49339277 -0.25378666  0.62405988  0.
  0.          0.          0.          0.        ]

Initial P matrix:
[[0.9  0.   0.   0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.9  0.   0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.9  0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.9  0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.   0.9  0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0

In [5]:
q3.policy_iteration()

Optimal value:
[ 3.08957523 -0.12784319  4.65641368  0.72389037  5.34413534  3.91325831
  3.91325831  4.65641368  3.91325831  5.34413534]

Optimal policy:
['Work', 'Work', 'Work', 'Work', 'Work', 'Reject', 'Reject', 'Accept', 'Reject', 'Accept']

Final R vector:
[ 0.19569346 -0.41561604  0.49339277 -0.25378666  0.62405988  0.
  0.          0.49339277  0.          0.62405988]

Final P matrix:
[[0.9  0.   0.   0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.9  0.   0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.9  0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.9  0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.   0.9  0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.9  0.   0.   0.02 0.02 0.02 0.02 0.02]
 [0.   0.   0.   0.   0.   0.2  0.2  0.2  0.2  0.2 ]
 [0.   0.   0.   0.   0.9  0.02 0.02 0.02 0.02 0.02]]


## Question 4

**Two-Stores Inventory Control.** 
We extend the capacity-constrained inventory example implemented in [rl/chapter3/simple_inventory_mdp_cap.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter3/simple_inventory_mdp_cap.py) as a `FiniteMarkovDecisionProcess` (the Finite MDP model for the capacity-constrained inventory example is described in detail in Chapters 1 and 2 of the RLForFinanceBook). Here we assume that we have two different stores, each with their own separate capacities $C_1$ and $C_2$, their own separate Poisson probability distributions of demand (with means $\lambda_1$ and $\lambda_2$), their own separate holding costs $h_1$ and $h_2$, and their own separate stockout costs $p_1$ and $p_2$. At 6pm upon stores closing each evening, each store can choose to order inventory from a common supplier (as usual, ordered inventory will arrive at the store 36 hours later). We are also allowed to transfer inventory from one store to another, and any such transfer happens overnight, i.e., will arrive by 6am next morning (since the stores are fairly close to each other). Note that the orders are constrained such that following the orders on each evening, each store's inventory position (sum of on-hand inventory and on-order inventory) cannot exceed the store's capacity (this means the action space is constrained to be finite). Each order made to the supplier incurs a fixed transportation cost of $K_1$ (fixed-cost means the cost is the same no matter how many units of non-zero inventory a particular store orders). Moving any non-zero inventory between the two stores incurs a fixed transportation cost of $K_2$. 

Model this as a derived class of `FiniteMarkovDecisionProcess` much like we did for `SimpleInventoryMDPCap` in the code repo. Set up instances of this derived class for different choices of the problem parameters (capacities, costs etc.), and determine the Optimal Value Function and Optimal Policy by invoking the function `value_iteration` (or `policy_iteration`) from file [rl/dynamic_programming.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/dynamic_programming.py).

Analyze the obtained Optimal Policy and verify that it makes intuitive sense as a function of the problem parameters.